In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
# Load data
externals_df = pd.read_csv("params/externals.csv")
aadt_df = pd.read_csv("data/AADTHistory 2023.xlsx - UnroundedAADT2023.csv", low_memory=False)
external_segment_link = pd.read_csv("params/externals-segments-link.csv")

In [3]:
# Melt AADT columns
aadt_year_cols = [col for col in aadt_df.columns if re.match(r"AADT\d{4}$", col)]
aadt_melted = aadt_df.melt(
    id_vars=["STATION", "RouteID", "BeginPoint", "EndPoint", "Section_Length", "DESC"],
    value_vars=aadt_year_cols,
    var_name="year",
    value_name="AADT"
)
aadt_melted["year"] = aadt_melted["year"].str.extract(r"(\d{4})").astype(int)

# Extract route number
aadt_melted["route"] = aadt_melted["RouteID"].str.extract(r"^(\d{4})")[0].dropna().astype(int)
aadt_melted = aadt_melted[['route','BeginPoint','EndPoint','year','AADT']]
aadt_melted.columns =['route','mp_begin','mp_end','year','AADT']
aadt_melted

,route,mp_begin,mp_end,year,AADT
0,6.0,0.000,46.0380,2023,457.0
1,6.0,46.038,77.5560,2023,409.0
2,6.0,77.556,82.8970,2023,586.0
3,6.0,82.897,83.9110,2023,2189.0
4,6.0,83.911,87.6940,2023,4012.0
...,...,...,...,...,...
196376,3468.0,0.000,5.1248,1981,0.0
196377,3469.0,0.000,6.9300,1981,0.0
196378,3470.0,0.000,1.0394,1981,0.0
196379,3478.0,0.000,2.0404,1981,0.0


In [4]:
# get external route, mp
external_segment_link[['route', 'milepost']] = external_segment_link['segid'].str.split('_', expand=True)
external_segment_link['route'] = external_segment_link['route'].astype(int)
external_segment_link['milepost'] = external_segment_link['milepost'].astype(float)
external_segment_link

,externalid,segid,route,milepost
0,3601,1082_000.0,1082,0.00
1,3602,0013_006.5,13,6.50
2,3603,1112_000.0,1112,0.00
3,3604,0015_368.1,15,368.10
4,3605,0038_003.2,38,3.20
5,3606,0091_010.1,91,10.10
6,3607,3462_002.8,3462,2.80
7,3608,0039_008.7,39,8.70
8,3609,0084_087.8,84,87.80
9,3610,2688_005.5,2688,5.50


In [5]:
merged_df = external_segment_link.merge(
    aadt_melted,
    on=['route']
).query("mp_begin < milepost + .01 <= mp_end")
merged_df = merged_df[['externalid','segid','year','AADT']]
merged_df = merged_df[merged_df['AADT']>0]
merged_df

,externalid,segid,year,AADT
0,3601,1082_000.0,2023,817.0
4,3601,1082_000.0,2022,783.0
8,3601,1082_000.0,2021,792.0
12,3601,1082_000.0,2020,739.0
16,3601,1082_000.0,2019,740.0
...,...,...,...,...
34780,3629,1826_004.9,2011,2555.0
34783,3629,1826_004.9,2010,205.0
34786,3629,1826_004.9,2009,200.0
34789,3629,1826_004.9,2008,195.0


In [6]:
merged_df.to_csv('intermediate/external-historic-aadt.csv', index=False)